# Kili Tutorial: How to query using the API

In this tutorial, we will show **query useful information through Kili's API**, interacting directly with the database.

There are 6 different types of data you could be interested in querying, and it is higly customizable : 
1. Your organization's info
2. The organization's users
3. KPI's and labeling data for different projectUsers
4. The whole project or selected parts
5. The assets themselves
6. Obviously, the labels

All those queries request, among other arguments, a `skip` and `first` argument, that control the number of results retrieved. `skip`is the rank at which the result start, and `first` is the extent of the query.

Additionally, for an overview of Kili, visit the [website](https://kili-technology.com), you can also check out the Kili [documentation](https://cloud.kili-technology.com/docs), or some other recipes.

For more information, the architecture of our API is available [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/), all fields are precisely detailled and can be queried, except those related to authentication.
Let's sign in, [create a project](), [upload some assets](https://github.com/kili-technology/kili-playground/blob/master/recipes/import_assets.ipynb) and [import predictions](https://github.com/kili-technology/kili-playground/blob/master/recipes/import_predictions.ipynb), if you need precisions on the steps carried-out below, you can refer to the mentionned recipes.

We now have a project ready, which information is in a `project_example` dictionnary, and which data we are going to query.

In [ ]:
#!pip install kili 
from kili.client import Kili
import os

api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT')
kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

project_example = {
    'title': 'Porsche or Tesla recognition',
    'description': 'Identify and locate cars',
    'input_type': 'IMAGE',
    'json_interface': {
        "jobs": {
            "JOB_0": {
                "mlTask": "OBJECT_DETECTION",
                "tools": [
                    "rectangle"
                ],
                "instruction": "What car brand ?",
                "required": 1,
                "isChild": False,
                "content": {
                    "categories": {
                        "TESLA": {"name": "Tesla"},
                        "FERRARI": {"name": "Ferrari"}
                    },
                    "input": "radio"
                }
            }
        }
    },

    'assets_to_import': [
        "https://images.caradisiac.com/logos/3/8/6/7/253867/S0-tesla-enregistre-d-importantes-pertes-au-premier-trimestre-175948.jpg",
        "https://img.sportauto.fr/news/2018/11/28/1533574/1920%7C1280%7Cc096243e5460db3e5e70c773.jpg"],

    'json_response': {
        "JOB_0": {
            "annotations": [{
                "boundingPoly": [{
                    "normalizedVertices": [
                        {"x": 0.16, "y": 0.82},
                        {"x": 0.16, "y": 0.32},
                        {"x": 0.82, "y": 0.32},
                        {"x": 0.82, "y": 0.82}
                    ]}
                ],
                "categories": [{"name": "TESLA", "confidence": 100}],
            }]
        }
    },
    'model_name': 'car-brand-localisation-v0.0.1'
}
project = kili.create_project(
    title=project_example['title'],
    description=project_example['description'],
    input_type=project_example['input_type'],
    json_interface=project_example['json_interface']
)
kili.append_many_to_dataset(
    project_id=project['id'],
    content_array=project_example['assets_to_import'],
    external_id_array=['ex1', 'ex2'])
kili.create_predictions(
    project_id=project['id'],
    external_id_array=['ex1'],
    model_name_array=[project_example['model_name']],
    json_response_array=[project_example['json_response']])

## 1. Organization info
We can query the information for the organization for specific users, and check that those parameters are correctly configured, either by providing 
- the `id` of the organization
- a specific `email`

Another argument is the `fields` that need to be returned, that can be customized using our graphQL doc [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#organization)

In [ ]:
email = os.getenv('KILI_USER_EMAIL')
fields = ['id', 'name','address','country','zipCode','city']
kili.organizations(email=email, fields=fields, first=3, skip= 0)

## 2. User info
We can query the information for an user, and check that those parameters are correctly configured, either by providing 
- the `email` of the user
- the `organization_id` of the users

Another argument is the `fields` that need to be returned, that can be customized using our graphQL doc [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#user). 
Notice that the **nested requests** are possible, to retrieve the information at the organization's level.

In [ ]:
fields =['id','activated','createdAt','email','name', 'organization.name', 'organization.city','organizationRole','updatedAt']
kili.users(email=email, fields=fields, skip=0, first=10)

## 3. ProjectUser info
We can query the information for a projectUser, and check that those parameters are correctly configured, either by providing 
- the `email` of the projectUser
- the `id` of the organization
- the `id` of the project
- the `id` of the projectUser

Another argument is the `fields` that need to be returned, that can be customized using our graphQL doc [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#projectuser). 
Notice that the **nested requests** are possible, to retrieve the information at the user's level, and hence at the organization's level, always with the same syntax :

In [ ]:
fields=['id','activated','consensusMark',"honeypotMark","numberOfAnnotations","numberOfLabels",'role',
        'user.email', 'user.name', 'user.organization.name', 'user.organization.zipCode']
kili.project_users(project_id=project['id'], fields=fields, first=10, skip=0)

## 4. Project info
We can query the information for a project, and check that those parameters are correctly configured, either by providing 
- the `email` of the projectUser
- a part of the title of the description `search_query`

Another argument is the `fields` that need to be returned, that can be customized using our graphQL doc [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#project). 
Notice that the **nested requests** are possible, to retrieve the information at the projectUser's level, via the `role subfield`, and hence, once again, at the user or the organization's level. It is also possible to query the information on the `author` of the project, always with the same syntax :

In [ ]:
fields = ['id','author.name','author.email', 'author.organization.name','consensusMark', 'createdAt',
          'description','honeypotMark', 'id', 'inputType', 'interfaceCategory', 'jsonInterface', 
          'maxWorkerCount', 'minConsensusSize', 'numberOfAssets', 'numberOfRemainingAssets', 
          'roles.id','roles.consensusMark', 'roles.numberOfAnnotations', 'roles.numberOfLabeledAssets', 
          'roles.numberOfLabels', 'roles.totalDuration', 'roles.user.email', 'roles.user.id', 'roles.user.name', 
          'title','titleAndDescription','useHoneyPot']
title = kili.projects(project_id=project['id'], fields=fields, first=10, skip=0)[0]['title']
kili.projects(project_id=project['id'], fields=fields, first=10, skip=0)

## 5. Labels info
We can query the information of labels, in order to use it later in our pipeline, either by providing 
- the `id`of the label or the asset to query
- the `id` of the project
- a part of the title or the description `search_query`
- many other parameters to filter the list, the details can be found [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#label), among which a part of the `externalId` of the asset, the status of the asset, the honeypot mark,etc... 

Another argument is the `fields` that need to be returned, that can be customized using our graphQL doc [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#label). 
Notice that the **nested requests** are possible, to retrieve the information at the projectUser's level, via the `role subfield`, and hence, once again, at the user or the organization's level. It is also possible to query the information on the `author` of the project, always with the same syntax :

In [ ]:
fields = ['id','honeypotMark','numberOfAnnotations','jsonResponse','labelType','skipped','createdAt',
        'author.email', 'author.name', 'author.organization.name', 'author.organization.zipCode']
kili.labels(project_id=project['id'], fields=fields, first=10, skip=0)

## 6. Assets info
We can query the information of assets, along with the labels or more info, either by providing 
- the `id`of the asset to query
- the `id` of the project
- a part of the title or the description `search_query`
- many other parameters to filter the list, the details can be found [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#asset), among which a part of the `externalId`, the status, the consensus level,etc... 

Another argument is the `fields` that need to be returned, that can be customized using our graphQL doc [here](https://cloud.kili-technology.com/docs/python-graphql-api/graphql-api/#asset). 
Notice that the **nested requests** are possible, to retrieve the information at the projectUser's level, via the `role subfield`, and hence, once again, at the user or the organization's level. It is also possible to query the information on the `author` of the project, always with the same syntax :

In [ ]:
fields=['id', 'honeypotMark', 'project.jsonInterface','content', 'createdAt', 'priority',
        'project.author.email', 'project.author.name', 'labels.jsonResponse', 'status',
        'labels.author.email', 'labels.author.organization.name', 'project.inputType']
kili.assets(project_id=project['id'], fields=fields, first=10, skip=0)

In [ ]:
assert project_example['title'] == title